# 🧭 정적 웹 크롤링 실습
전시 산업 참가기업 정보를 `requests`와 `BeautifulSoup`을 사용해 수집하고, 출력 및 CSV로 저장해보는 실습입니다.

## 📦 필요한 라이브러리 설치

In [1]:
!pip install requests beautifulsoup4 pandas

  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)


## 🔗 1단계: 웹페이지 요청하기
`requests` 라이브러리를 사용해 HTML을 가져옵니다.

#### 📌 알아두기: `requests.get()`이 하는 일
- 웹사이트에 **"나 이 페이지 좀 보여줘!"** 라는 요청을 보냅니다.
- 우리가 흔히 브라우저에서 주소창에 입력하는 행동과 같은 일이죠!
- 받은 응답은 `response` 변수에 저장되고, 그 안에는 HTML 코드가 담겨 있어요.

#### ❗ 왜 `headers`를 써요?
- 일부 사이트는 크롤러가 접근하면 막는 경우가 있어요.
- `"User-Agent"`를 넣으면 **"사람이 브라우저로 보는 것처럼"** 속여서 막히지 않게 해주는 역할이에요.

In [2]:
import requests

url = "https://www.interzoo.com/en/exhibitor-list"
headers = {
    "User-Agent": "Mozilla/5.0"
}
response = requests.get(url, headers=headers)

# 응답 확인
print("응답 코드:", response.status_code)
print("HTML 일부:", response.text[:500])  # 일부만 출력


응답 코드: 200
HTML 일부: 


<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en">
<!--<![endif]-->
<head>
<link href="https://mc-e5b0d581-4409-4340-bc8b-9266-cdn-endpoint.azureedge.net/-/media/feature/experience-accelerator/bootstrap-4/bootstrap-4/styles/optimized-min.css?rev=fb896899


## 🧱 2단계: HTML 구조 분석 및 파싱
`BeautifulSoup`을 사용해 HTML을 구조화된 형태로 파싱합니다.

> `response.text`는 단순한 HTML 텍스트입니다. 구조는 있지만, 우리가 직접 `<div>`나 `<h3>` 같은 태그를 추출해서 사용하기엔 불편하죠.

> 그래서 `BeautifulSoup`을 사용해 `soup` 객체로 변환하면 HTML을 **나무 구조(tree)**처럼 다룰 수 있게 되고, 원하는 요소를 `soup.select()`나 `soup.find()`로 쉽게 꺼낼 수 있게 됩니다.

In [4]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text, "html.parser")

# HTML 구조 일부 보기
print(soup.prettify()[:500])


<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en">
 <!--<![endif]-->
 <head>
  <link href="https://mc-e5b0d581-4409-4340-bc8b-9266-cdn-endpoint.azureedge.net/-/media/feature/experience-accelerator/bootstrap-4/bootstrap-4/styles/optimized-min.css?rev=fb896899ea424d2192


## 🏢 3단계: 기업 정보 리스트 추출하기
HTML을 분석하여 기업명, 부스정보, 배지 등을 추출합니다.

In [ ]:
exhibitors = soup.select("a[href*='/exhibitors/']")

results = []

for card in exhibitors:
    name = card.select_one("h3")
    booth = card.select_one("span[aria-label$='Stand.']")
    badges = [badge.text.strip() for badge in card.select("div.badges span")]
    link = card['href']

    results.append({
        "name": name.text.strip() if name else "N/A",
        "booth": booth.text.strip() if booth else "N/A",
        "badges": ", ".join(badges),
        "link": "https://www.interzoo.com" + link
    })

# 일부 출력
for r in results[:5]:
    print(r)


## 💾 4단계: 결과를 CSV로 저장하기

In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df.to_csv("interzoo_exhibitors.csv", index=False)
print("CSV 저장 완료!")

## 🚀 확장 학습 팁
- `requests`로 페이징 처리 응용하기
- `bs4`로 이미지, 링크 등 추가 수집
- Pandas로 정리 후 Excel 저장까지 확장